## 1. Setup & Installation

In [1]:
"""Disable Unsloth interference and install dependencies."""
import os

# Disable torch compile to avoid Unsloth interference
os.environ["TORCHDYNAMO_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"] = "1"

# Install required dependencies
!pip install -q transformers datasets peft accelerate bitsandbytes
!pip install -q emoji rouge-score huggingface_hub
!pip install -q sentencepiece

In [2]:
"""Import libraries and configure environment."""
import re
import logging
from typing import Dict, List, Optional

import emoji
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

2026-01-16 11:33:05.455265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768563185.477604     147 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768563185.484501     147 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768563185.501665     147 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768563185.501687     147 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768563185.501689     147 computation_placer.cc:177] computation placer alr

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.83 GB


## 2. Configuration

In [ ]:
"""Configuration constants for training."""

HF_TOKEN = ""  
HF_USERNAME = ""
HF_REPO_NAME = ""git 


DATA_PATH = "/kaggle/input/suummarization-data/training_dataset_openrouter.csv"

# Model settings
MODEL_NAME = "google/flan-t5-large"
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 256

# LoRA settings
LORA_R = 64
LORA_ALPHA = 128
LORA_DROPOUT = 0.05

# Training settings
BATCH_SIZE = 4  # Reduced for stability
GRADIENT_ACCUMULATION_STEPS = 8  # Effective batch = 32
LEARNING_RATE = 0.0001
NUM_EPOCHS = 20
SAVE_STEPS = 173
EVAL_STEPS = 173

# Output directory
OUTPUT_DIR = "/kaggle/working/checkpoints"

# Computed values
HF_MODEL_ID = f"{HF_USERNAME}/{HF_REPO_NAME}"

In [4]:
"""Login to HuggingFace Hub."""
login(token=HF_TOKEN)
logger.info(f"Logged in to HuggingFace Hub as {HF_USERNAME}")
logger.info(f"Model will be pushed to: {HF_MODEL_ID}")

## 3. Data Loading & Preprocessing

In [5]:
"""Text preprocessing functions."""


def remove_markdown_formatting(text: str) -> str:
    """Remove markdown bold and italic formatting."""
    text = re.sub(r"\*\*(.+?)\*\*", r"\1", text)
    text = re.sub(r"__(.+?)__", r"\1", text)
    text = re.sub(r"\*(.+?)\*", r"\1", text)
    text = re.sub(r"_(.+?)_", r"\1", text)
    return text


def remove_emojis(text: str) -> str:
    """Remove emojis from text."""
    return emoji.replace_emoji(text, replace="")


def normalize_unicode(text: str) -> str:
    """Normalize unicode characters to ASCII equivalents."""
    replacements = {
        "—": "-", "–": "-", "'": "'", "'": "'",
        "\"": '"', "\"": '"', "…": "...",
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text


def clean_whitespace(text: str) -> str:
    """Clean excessive whitespace and newlines."""
    text = text.replace("\n", " ").replace("\r", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def clean_text(text: str) -> str:
    """Apply all text cleaning steps."""
    if not isinstance(text, str):
        return ""
    text = remove_markdown_formatting(text)
    text = remove_emojis(text)
    text = normalize_unicode(text)
    text = clean_whitespace(text)
    return text

In [6]:
"""Test preprocessing functions."""

test_cases = [
    "**STRIKE NOW!** 🚀 Post at 7PM—don't miss it!",
    "This is __important__ and *urgent* info.",
    "Peak engagement... score: 95—the best!",
]

print("Preprocessing Test Results:")
print("=" * 50)
for test in test_cases:
    cleaned = clean_text(test)
    print(f"Input:  {test}")
    print(f"Output: {cleaned}")
    print("-" * 50)

Preprocessing Test Results:
Input:  **STRIKE NOW!** 🚀 Post at 7PM—don't miss it!
Output: STRIKE NOW! Post at 7PM-don't miss it!
--------------------------------------------------
Input:  This is __important__ and *urgent* info.
Output: This is important and urgent info.
--------------------------------------------------
Input:  Peak engagement... score: 95—the best!
Output: Peak engagement... score: 95-the best!
--------------------------------------------------


In [7]:
"""Load and preprocess the dataset."""


def load_and_preprocess_data(file_path: str) -> pd.DataFrame:
    """Load CSV and apply preprocessing."""
    logger.info(f"Loading data from {file_path}")
    df = pd.read_csv(file_path)

    original_count = len(df)
    logger.info(f"Loaded {original_count} rows")

    # Apply preprocessing
    logger.info("Applying text preprocessing...")
    df["input_text"] = df["input_text"].apply(clean_text)
    df["target_text"] = df["target_text"].apply(clean_text)

    # Remove empty rows
    df = df[
        (df["input_text"].str.len() > 0) &
        (df["target_text"].str.len() > 0)
    ]

    cleaned_count = len(df)
    logger.info(f"After cleaning: {cleaned_count} rows")

    return df


# Load data
df = load_and_preprocess_data(DATA_PATH)

# Display samples
print("\nSample Data:")
for i in range(2):
    print(f"Input: {df.iloc[i]['input_text']}")
    print(f"Target: {df.iloc[i]['target_text'][:80]}...")
    print("-" * 50)


Sample Data:
Input: Day: Monday, Time: 09:00 - 12:00, Score: 97, Dominance: Unrivaled, Shape: Sustained Plateau, Style: Storytelling (Narrative flow)
Target: Monday mid-mornings capture professionals settling into their workweek, seeking ...
--------------------------------------------------
Input: Day: Thursday, Time: 13:00 - 17:00, Score: 85, Dominance: Unrivaled, Shape: Sustained Plateau, Style: Strategic (Focus on ROI & Growth)
Target: Thursday afternoon (13:00-17:00) delivers exceptional ROI with an 85/100 score-n...
--------------------------------------------------


In [8]:
"""Create train/validation split."""


def create_dataset_splits(
    df: pd.DataFrame,
    val_ratio: float = 0.1,
    seed: int = 42
) -> DatasetDict:
    """Split data into train and validation sets."""
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    val_size = int(len(df) * val_ratio)
    train_df = df[val_size:]
    val_df = df[:val_size]

    logger.info(f"Train: {len(train_df)}, Val: {len(val_df)}")

    train_dataset = Dataset.from_pandas(train_df[["input_text", "target_text"]])
    val_dataset = Dataset.from_pandas(val_df[["input_text", "target_text"]])

    return DatasetDict({"train": train_dataset, "validation": val_dataset})


dataset = create_dataset_splits(df)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5508
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 611
    })
})


## 4. Model & Tokenizer Setup

In [9]:
"""Load tokenizer."""

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
logger.info(f"Tokenizer loaded: {MODEL_NAME}")

In [10]:
"""Load model with 4-bit quantization (QLoRA)."""

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    dtype=torch.float16,
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=False  # Disable to avoid Unsloth conflict
)

logger.info(f"Model loaded with 4-bit quantization: {MODEL_NAME}")

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens', 'encoder.embed_tokens']
  warnings.warn(


In [11]:
"""Configure and apply LoRA."""

# LoRA configuration for T5
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=["q", "v", "k", "o"],
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 37,748,736 || all params: 820,898,816 || trainable%: 4.5985


In [12]:
"""Tokenization functions."""

T5_PREFIX = "summarize : "


def preprocess_function(examples: Dict) -> Dict:
    """Tokenize input and target text for T5."""
    inputs = [T5_PREFIX + text for text in examples["input_text"]]
    targets = examples["target_text"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        padding="max_length",
        truncation=True,
    )

    labels = tokenizer(
        text_target=targets,
        max_length=MAX_TARGET_LENGTH,
        padding="max_length",
        truncation=True,
    )

    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in ids]
        for ids in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


logger.info("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["input_text", "target_text"],
    desc="Tokenizing",
)

print(tokenized_dataset)

Tokenizing:   0%|          | 0/5508 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/611 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5508
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 611
    })
})


## 5. Training Setup

In [13]:
"""Reset training dari awal - Hapus lokal + Hub.

JALANKAN CELL INI HANYA JIKA INGIN RESET TOTAL!
Set RESET_TRAINING = True untuk reset, False untuk skip.
"""
import shutil
from huggingface_hub import HfApi, list_repo_files

RESET_TRAINING = False  # Set True untuk reset, False untuk skip


def reset_local_checkpoints(output_dir: str) -> None:
    """Hapus semua checkpoint lokal.

    Args:
        output_dir: Path ke direktori checkpoint.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        logger.info("Local checkpoints deleted")
    else:
        logger.info("No local checkpoints to delete")


def reset_hub_repository(repo_id: str, token: str) -> None:
    """Hapus semua files di HuggingFace Hub repository.

    Args:
        repo_id: ID repository di Hub (username/repo-name).
        token: HuggingFace API token.
    """
    api = HfApi(token=token)
    try:
        files = list_repo_files(repo_id, token=token)
        for file in files:
            if file != ".gitattributes":
                api.delete_file(
                    path_in_repo=file,
                    repo_id=repo_id,
                    token=token
                )
                logger.info(f"Deleted from Hub: {file}")
        logger.info("HuggingFace Hub reset complete")
    except Exception as e:
        logger.warning(f"Hub reset skipped: {e}")


if RESET_TRAINING:
    reset_local_checkpoints(OUTPUT_DIR)
    reset_hub_repository(HF_MODEL_ID, HF_TOKEN)
    logger.info("Training will start from step 0")
else:
    logger.info("Reset skipped - will resume from checkpoint if available")

In [14]:
"""Auto-load checkpoint dari HuggingFace Hub jika lokal kosong.

Ini memastikan training bisa resume dengan benar setelah Kaggle session restart.
"""
from huggingface_hub import snapshot_download, list_repo_files


def get_local_checkpoints(output_dir: str) -> list:
    """Get list of local checkpoint directories.

    Args:
        output_dir: Path ke direktori checkpoint.

    Returns:
        List of checkpoint directory names.
    """
    if not os.path.exists(output_dir):
        return []
    return [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]


def sync_from_hub(
    repo_id: str,
    output_dir: str,
    token: str
) -> None:
    """Download model files dari Hub jika lokal kosong.

    Args:
        repo_id: ID repository di Hub.
        output_dir: Path untuk menyimpan model.
        token: HuggingFace API token.
    """
    local_checkpoints = get_local_checkpoints(output_dir)

    if local_checkpoints:
        logger.info(f"Found {len(local_checkpoints)} local checkpoints")
        return

    try:
        hub_files = list_repo_files(repo_id, token=token)
        if "adapter_model.safetensors" in hub_files:
            logger.info("Downloading model from HuggingFace Hub...")
            snapshot_download(
                repo_id=repo_id,
                local_dir=output_dir,
                token=token,
                ignore_patterns=["*.md", "*.txt"],
            )
            logger.info(f"Model downloaded to: {output_dir}")
        else:
            logger.info("No model found on Hub - starting fresh")
    except Exception as e:
        logger.warning(f"Could not sync from Hub: {e}")
        logger.info("Starting fresh training")


sync_from_hub(HF_MODEL_ID, OUTPUT_DIR, HF_TOKEN)

In [15]:
"""Configure training arguments dengan learning rate scheduler."""

training_args = Seq2SeqTrainingArguments(
    # Output & Hub
    output_dir=OUTPUT_DIR,
    push_to_hub=True,
    hub_model_id=HF_MODEL_ID,
    hub_strategy="checkpoint",
    hub_token=HF_TOKEN,

    # Training
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    optim="paged_adamw_8bit",

    # Learning Rate Scheduler
    lr_scheduler_type="cosine",  # Cosine decay - smooth decrease
    warmup_ratio=0.1,            # 10% steps untuk warmup

    # Precision
    fp16=False,
    bf16=False,

    # Saving & Evaluation
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=100,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    # Logging
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=50,
    report_to="none",

    # Generation
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,

    # Gradient
    gradient_checkpointing=False,
    max_grad_norm=1.0,
)

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
)

logger.info("Training arguments configured with cosine LR scheduler")
logger.info(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
logger.info(f"Warmup: 10% of total steps")

In [16]:
"""Check for existing checkpoint to resume from."""


def get_latest_checkpoint(output_dir: str) -> Optional[str]:
    """Find the latest checkpoint directory."""
    if not os.path.exists(output_dir):
        return None

    checkpoints = [
        d for d in os.listdir(output_dir)
        if d.startswith("checkpoint-")
    ]

    if not checkpoints:
        return None

    checkpoints.sort(key=lambda x: int(x.split("-")[1]))
    latest = os.path.join(output_dir, checkpoints[-1])
    return latest


latest_checkpoint = get_latest_checkpoint(OUTPUT_DIR)
if latest_checkpoint:
    logger.info(f"Found checkpoint: {latest_checkpoint}")
else:
    logger.info("No checkpoint found. Starting fresh.")

In [17]:
"""Initialize trainer."""

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

logger.info("Trainer initialized")

/tmp/ipykernel_147/3246972690.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


## 6. Training

In [18]:
    """Start or resume training."""
    
    logger.info("=" * 50)
    logger.info("Starting Training")
    logger.info("=" * 50)
    logger.info(f"Model: {MODEL_NAME}")
    logger.info(f"Train: {len(tokenized_dataset['train'])}, Val: {len(tokenized_dataset['validation'])}")
    logger.info(f"Epochs: {NUM_EPOCHS}, Hub: {HF_MODEL_ID}")
    logger.info("=" * 50)
    
    # Train
    train_result = trainer.train(resume_from_checkpoint=latest_checkpoint)
    
    # Log metrics
    metrics = train_result.metrics
    logger.info(f"Training completed. Final metrics: {metrics}")

Step,Training Loss,Validation Loss


## 7. Evaluation

In [19]:
"""Evaluate on validation set."""

eval_results = trainer.evaluate()
print("\nEvaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


Evaluation Results:
eval_loss: 0.8693
eval_runtime: 74.3001
eval_samples_per_second: 8.2230
eval_steps_per_second: 2.0590
epoch: 20.0000


In [20]:
"""Generate sample predictions."""


def generate_prediction(input_text: str) -> str:
    """Generate prediction for a single input."""
    full_input = T5_PREFIX + input_text
    inputs = tokenizer(
        full_input,
        return_tensors="pt",
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_length=MAX_TARGET_LENGTH,
        num_beams=4,
        early_stopping=True,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


print("Sample Predictions:")
print("=" * 50)

for i in range(3):
    sample = dataset["validation"][i]
    prediction = generate_prediction(sample["input_text"])

    print(f"\n--- Sample {i + 1} ---")
    print(f"Input: {sample['input_text']}")
    print(f"Expected: {sample['target_text'][:100]}...")
    print(f"Predicted: {prediction}")

Sample Predictions:

--- Sample 1 ---
Input: Day: Tuesday, Time: 22:00 - 00:00, Score: 73, Dominance: Unrivaled, Shape: Sustained Plateau, Style: Urgent (Creating FOMO/Action)
Expected: STRIKE NOW: Tuesday late night (22:00-00:00) crushes competitors with a 73/100 score-26 points ahead...
Predicted: Don't miss this window! Tuesday late night (22:00-00:00) crushes competitors with a 73/100 score-nearly double the runner-up. Night owls are actively scrolling before bed, creating a sustained engagement plateau you can't afford to miss.

--- Sample 2 ---
Input: Day: Thursday, Time: 09:00 - 12:00, Score: 97, Dominance: Clear Lead, Shape: Sustained Plateau, Style: Advisory (Consultative, Helpful)
Expected: Thursday mid-mornings (9am-12pm) capture professionals during their most active work breaks, with a ...
Predicted: Thursday mid-morning (9 AM-12 PM) captures professionals during their first work break, delivering a commanding 97/100 engagement score-significantly outperforming alternative

## 8. Export for Inference

In [21]:
"""Save merged model for inference."""

merged_model = model.merge_and_unload()

MERGED_OUTPUT_DIR = f"{OUTPUT_DIR}/merged"
merged_model.save_pretrained(MERGED_OUTPUT_DIR)
tokenizer.save_pretrained(MERGED_OUTPUT_DIR)

logger.info(f"Merged model saved to: {MERGED_OUTPUT_DIR}")

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [26]:
"""Skip ONNX export - model 4-bit tidak compatible dengan ONNX."""

logger.info("=" * 50)
logger.info("ONNX export skipped")
logger.info("=" * 50)
logger.info("Reason: 4-bit quantized models are not compatible with ONNX")
logger.info("")
logger.info("For optimized inference, use BetterTransformer:")
logger.info("  model = model.to_bettertransformer()")
logger.info("")
logger.info("Available inference options:")
logger.info(f"  1. LoRA adapter: {HF_MODEL_ID}")
logger.info(f"  2. Merged model: {HF_MODEL_ID}/merged")

# Set to None so upload cell knows to skip
ONNX_OUTPUT_DIR = None

In [27]:
"""Usage instructions."""

print("Done! Model is ready for inference.")
print(f"\nTo load the LoRA adapter model:")
print(f"")
print(f"from transformers import AutoModelForSeq2SeqLM, AutoTokenizer")
print(f"from peft import PeftModel")
print(f"")
print(f'base_model = AutoModelForSeq2SeqLM.from_pretrained("{MODEL_NAME}")')
print(f'model = PeftModel.from_pretrained(base_model, "{HF_MODEL_ID}")')
print(f'tokenizer = AutoTokenizer.from_pretrained("{HF_MODEL_ID}")')

Done! Model is ready for inference.

To load the LoRA adapter model:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = PeftModel.from_pretrained(base_model, "raflisbk/t5-posting-time-summarizer")
tokenizer = AutoTokenizer.from_pretrained("raflisbk/t5-posting-time-summarizer")


In [28]:
"""Save dan upload semua model ke HuggingFace Hub."""
from huggingface_hub import HfApi


def upload_folder_to_hub(
    local_path: str,
    repo_id: str,
    path_in_repo: str,
    token: str
) -> None:
    """Upload folder ke HuggingFace Hub.

    Args:
        local_path: Path lokal folder yang akan di-upload.
        repo_id: ID repository di Hub.
        path_in_repo: Path tujuan di dalam repository.
        token: HuggingFace API token.
    """
    api = HfApi(token=token)
    api.upload_folder(
        folder_path=local_path,
        repo_id=repo_id,
        path_in_repo=path_in_repo,
        token=token,
    )
    logger.info(f"Uploaded {local_path} to {repo_id}/{path_in_repo}")


# Save model locally first
trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

# 1. Upload LoRA adapter (root)
logger.info("Uploading LoRA adapter...")
trainer.push_to_hub(commit_message="Final LoRA adapter", blocking=True)

# 2. Upload merged model
logger.info("Uploading merged model...")
upload_folder_to_hub(
    local_path=MERGED_OUTPUT_DIR,
    repo_id=HF_MODEL_ID,
    path_in_repo="merged",
    token=HF_TOKEN
)

# 3. Upload ONNX model (jika ada)
if ONNX_OUTPUT_DIR and os.path.exists(ONNX_OUTPUT_DIR):
    logger.info("Uploading ONNX model...")
    upload_folder_to_hub(
        local_path=ONNX_OUTPUT_DIR,
        repo_id=HF_MODEL_ID,
        path_in_repo="onnx",
        token=HF_TOKEN
    )
else:
    logger.info("ONNX model not available, skipping upload")
    
logger.info("All models uploaded successfully!")
logger.info(f"Repository: https://huggingface.co/{HF_MODEL_ID}")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
# Opsi 1: LoRA adapter (lebih kecil)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = PeftModel.from_pretrained(base_model, "raflisbk/t5-posting-time-summarizer")
tokenizer = AutoTokenizer.from_pretrained("raflisbk/t5-posting-time-summarizer")

# Opsi 2: Merged model (lebih cepat load)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "raflisbk/t5-posting-time-summarizer",
    subfolder="merged"
)

# Optimasi inference
model = model.to_bettertransformer()